# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
file_to_load = "../WeatherPy/output_data/cities.csv"
weather_df = pd.read_csv(file_to_load)
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,26.00,41,0,4.63,ZA,1612278813
1,1,tungor,53.3808,142.9564,-31.80,84,100,6.87,RU,1612278813
2,2,bluff,-46.6000,168.3333,14.41,90,56,2.14,NZ,1612278814
3,3,guanica,17.9716,-66.9080,27.78,63,18,3.13,PR,1612278814
4,4,hasaki,35.7333,140.8333,4.00,52,20,2.57,JP,1612278814
...,...,...,...,...,...,...,...,...,...,...
554,554,cap malheureux,-19.9842,57.6142,27.22,74,1,3.13,MU,1612278888
555,555,vila nova de milfontes,37.7238,-8.7828,16.67,59,31,2.68,PT,1612278888
556,556,ambovombe,-25.1667,46.0833,28.16,76,0,6.49,MG,1612278888
557,557,san quintin,30.4833,-115.9500,16.35,68,100,3.42,MX,1612278889


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)
# Use the Lat and Lng as locations 
locations = weather_df[["Lat", "Lng"]]
# and Humidity as the weight.
humidity = weather_df["Humidity"]

# Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities to fit weather conditions
## A max temperature lower than 80 degrees but higher than 70.
n_cities_df = weather_df[weather_df['Max Temperature'] >= 21]
n_cities_df = n_cities_df[weather_df['Max Temperature'] <= 27]
## Wind speed less than 10 mph.
n_cities_df = n_cities_df[weather_df['Wind Speed'] <= 10]
## Zero cloudiness
n_cities_df = n_cities_df[weather_df['Cloudiness'] == 0]
# Create new DataFrame fitting weather criteria
n_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,26.00,41,0,4.63,ZA,1612278813
12,12,salalah,17.0151,54.0924,24.00,57,0,1.54,OM,1612278815
22,22,pisco,-13.7000,-76.2167,23.00,73,0,4.63,PE,1612278816
54,54,puri,19.8000,85.8500,22.00,60,0,0.51,IN,1612278820
80,80,antofagasta,-23.6500,-70.4000,22.78,60,0,4.63,CL,1612278823
132,132,comodoro rivadavia,-45.8667,-67.5000,23.00,49,0,2.57,AR,1612278831
148,148,port elizabeth,-33.9180,25.5701,24.00,64,0,9.77,ZA,1612278688
195,195,gat,31.6100,34.7642,22.78,52,0,1.10,IL,1612278840
238,238,port alfred,-33.5906,26.8910,23.33,77,0,5.81,ZA,1612278846
244,244,luderitz,-26.6481,15.1594,21.40,67,0,6.84,NaN,1612278608


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.
hotel_df = n_cities_df[["City", "Lat", "Lng", "Country"]].reset_index()
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,index,City,Lat,Lng,Country,Hotel Name
0,0,bredasdorp,-34.5322,20.0403,ZA,
1,12,salalah,17.0151,54.0924,OM,
2,22,pisco,-13.7000,-76.2167,PE,
3,54,puri,19.8000,85.8500,IN,
4,80,antofagasta,-23.6500,-70.4000,CL,
5,132,comodoro rivadavia,-45.8667,-67.5000,AR,
6,148,port elizabeth,-33.9180,25.5701,ZA,
7,195,gat,31.6100,34.7642,IL,
8,238,port alfred,-33.5906,26.8910,ZA,
9,244,luderitz,-26.6481,15.1594,NaN,


In [6]:
# Set parameters to search for hotels with 5000 meters.
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Hit the Google Places API for each city's coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [7]:
# Store the first Hotel result into the DataFrame.

In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for {index}.: {city_name}")
    response = requests.get(base_url, params=params).json()
    response

    results = response['results']

   # Store the first Hotel result into the DataFrame.
    try:
        print(f"A hotel within 5000 meters of {city_name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No Hotel found. Skipping...")
        print("---------------------------")
    print("Ending Search...")

Retrieving Results for 0.: bredasdorp
A hotel within 5000 meters of bredasdorp is Victoria Hotel - Bredasdorp
Ending Search...
Retrieving Results for 1.: salalah
A hotel within 5000 meters of salalah is HYATT SALALAH HOTEL
Ending Search...
Retrieving Results for 2.: pisco
A hotel within 5000 meters of pisco is HOSTAL MIRAMAR
Ending Search...
Retrieving Results for 3.: puri
A hotel within 5000 meters of puri is MAYFAIR Heritage, Puri
Ending Search...
Retrieving Results for 4.: antofagasta
A hotel within 5000 meters of antofagasta is Enjoy Antofagasta Hotel Del Desierto
Ending Search...
Retrieving Results for 5.: comodoro rivadavia
A hotel within 5000 meters of comodoro rivadavia is Comodoro Hotel
Ending Search...
Retrieving Results for 6.: port elizabeth
A hotel within 5000 meters of port elizabeth is The King Edward Hotel
Ending Search...
Retrieving Results for 7.: gat
A hotel within 5000 meters of gat is Desert Gat
Ending Search...
Retrieving Results for 8.: port alfred
A hotel within

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
# Screenshots of the maps are saved in output_data folder